In [19]:
import functools
import time

# import tensorflow as tf
# import tensorflow_text

import seqio

from t5.evaluation import metrics
from t5.data import preprocessors
import t5

In [22]:
import pandas as pd

df = pd.read_csv('gs://cloud-ai-platform-2f444b6a-a742-444b-b91a-c7519f51bd77/llm/data/test/ul2-xsum-infer-test-samples.tsv', 
                 delimiter='\t', names=['article', 'highlights'])

In [23]:
df.head()

,article,highlights
0,(CNN)The attorney for a suburban New York card...,A lawyer for Dr. Anthony Moschetto says the ch...
1,"(CNN)When the earthquake hit, many of Nepal's ...",NaN
2,Sam Allardyce is already making plans for the ...,NaN
3,A 44-year-old woman has been shot dead in fron...,NaN
4,"With a net worth of £260million, Hollywood pow...",NaN


In [4]:
t5.data.preprocessors.preprocess_tsv?

Signature:
t5.data.preprocessors.preprocess_tsv(
    line,
    field_delim='\t',
    num_fields=2,
    inputs_format='{0}',
    targets_format='{1}',
)
Docstring:
Parse tab-delimited strings into inputs and targets.

This function takes a tf.data.Dataset of strings, each of which contains
tab-delimited fields.  The function returns a tf.data.Dataset of feature
dictionaries of the form {"inputs": string, "targets": string}.

inputs_format contains a template string and field numbers used to produce
the "inputs" string.
targets_format contains a template string and field numbers used to produce
the "targets" string.

Example:
  The input dataset contains the lines:
  "6,7,42"
  "2,9,18"
  preprocess_tsv(dataset,
                 field_delim=',',
                 inputs_format='numerator: {2} denominator: {1}',
                 targets_format='quotient: {0}'
  would produce a dataset containing the dictionaries:
  {"inputs": "numerator: 42 denomnator: 7", "targets": "quotient: 6"}
  {"inp

In [37]:
# seqio tokenization
DEFAULT_OUTPUT_FEATURES = {
    "inputs": seqio.Feature(
        vocabulary=t5.data.get_default_vocabulary(), add_eos=True,
        required=False),
    "targets": seqio.Feature(
        vocabulary=t5.data.get_default_vocabulary(), add_eos=True)
}

DEFAULT_PREPROCESSORS = [
    seqio.preprocessors.tokenize,
    seqio.CacheDatasetPlaceholder(),
    seqio.preprocessors.append_eos_after_trim,
]

paths = {
    'train': 'gs://cloud-ai-platform-2f444b6a-a742-444b-b91a-c7519f51bd77/llm/data/test/ul2-xsum-infer-test-samples.txt',
}

# Task definition
task = seqio.Task(
    'xsum',
    source=seqio.TextLineDataSource(
        split_to_filepattern=paths,
        skip_header_lines=0
    ),
  preprocessors=[
      functools.partial(
          t5.data.preprocessors.preprocess_tsv,
          field_delim="\t",
          num_fields=2,
          inputs_format="summarize: {0}",
          targets_format=""),
      *DEFAULT_PREPROCESSORS,
  ],
    metric_fns=[metrics.rouge],
    output_features=DEFAULT_OUTPUT_FEATURES)

In [38]:
dataset = task.get_dataset(sequence_length={'article':512}, 
                           split='train').as_numpy_iterator()

In [39]:
dataset.next()

InvalidArgumentError: Expect 2 fields but have 1 in record 0
	 [[{{node DecodeCSV}}]] [Op:IteratorGetNext]

In [14]:
dataset.next()['inputs']

array([21603,    10,  2759,   151,    30,     3,     9, 13057,  3777,
          84,   263,    46,  3583,  9501,    16,  5053,   130,  2833,
        1601,   227,  5272,     3,  2905,  6724,  1433,    84,    47,
           3,  2172,    57,    80, 10837,    12,     3,    31,   439,
          53,  7807,  8915,    95,     8,  6112,    11, 25341,    34,
          31,     5, 13057,   243,     3,    31,     9,   422,   381,
          13,   722,    31,    30, 16736,   204,  4450,    45,  1919,
          12,   368,  6604,  5735,    16,   368,  5092,     6, 25932,
          13, 25808,    11,   487,  4012,  5157,    11,   130,  7122,
          91,    57,  3583,  1035, 11225,     5,    71,  5053,     3,
       20804,  5468,     7, 15893,   243,   192,   151,   130,  1026,
          12,  9777,  2146,  4457,    28,   125,    33,  6141,    12,
          36,  4012,  5157,   227,     8,    73,  3992,  1259,  1361,
        9501,    44, 22227,  5735,    16,  5053,     5, 25731,   323,
          21,   671,

In [18]:
from datasets import load_dataset, load_metric
from tqdm import tqdm

dataset_cnn = load_dataset("ccdv/cnn_dailymail", '3.0.0')

for data_point_idx in tqdm(range(1, 11490, int(11490 / 20))):
    datapoint = dataset_cnn['test'][data_point_idx]
    print(datapoint)

Extracting data files: 100%|██████████| 5/5 [00:45<00:00,  9.00s/it]


Dataset cnn_dailymail downloaded and prepared to /home/jupyter/.cache/huggingface/datasets/ccdv___cnn_dailymail/3.0.0/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f. Subsequent calls will reuse this data.


100%|██████████| 21/21 [00:00<00:00, 4844.64it/s]

{'article': '(CNN)The attorney for a suburban New York cardiologist charged in what authorities say was a failed scheme to have another physician hurt or killed is calling the allegations against his client "completely unsubstantiated." Appearing Saturday morning on CNN\'s "New Day," Randy Zelin defended his client, Dr. Anthony Moschetto, who faces criminal solicitation, conspiracy, burglary, arson, criminal prescription sale and weapons charges in connection to what prosecutors called a plot to take out a rival doctor on Long Island. "None of anything in this case has any evidentiary value," Zelin told CNN\'s Christi Paul.  "It doesn\'t matter what anyone says, he is presumed to be innocent." Moschetto,54, pleaded not guilty to all charges Wednesday.  He was released after posting $2 million bond and surrendering his passport. Zelin said that his next move is to get Dr. Moshetto back to work. "He\'s got patients to see. This man, while he was in a detention cell, the only thing that h